In [1]:
import pandas as pd
import numpy as np
import os
import glob
import chardet
import joblib
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from imblearn.under_sampling import RandomUnderSampler

In [5]:
curr_path = 'C:/Users/cheng/Desktop/DSA3101/DSA3101_project/LM and DT model'
data_1989 = pd.read_csv("1989.csv", encoding="mac_roman") ## Change file according to the year you are interested in

In [7]:
## Function to group time
def bin_time(crs_time):
    if crs_time == 2400:
        return 3
    return crs_time // 600


## Functin to get season based on month 
def get_season(month):
    season_dict = {(1,2,3): 'spring', (4,5,6): 'summer', (7,8,9): 'autumn', (10,11,12): 'winter'}
    for key, val in season_dict.items():
        if month in key:
            return val

In [8]:
## Data preprocessing 
data_1989['SEASON'] = data_1989['MONTH'].apply(get_season)
data_1989['CRS_ARR_BIN'] = data_1989['CRS_ARR_TIME'].apply(bin_time)   
## Definition of delay is more than 60 mins
data_1989['HAS_ARR_DELAY'] = data_1989['ARR_DELAY_GROUP'].apply(lambda x: 1 if x > 3 else 0)

In [9]:
## Select columns 
variables = ['DAY_OF_WEEK', 'SEASON', 'DISTANCE', 'CRS_ARR_BIN', 'PRCP_ORIGIN', 'SNOW_ORIGIN', 
                        'SNWD_ORIGIN', 'TMAX_ORIGIN', 'TMIN_ORIGIN', 'HAS_ARR_DELAY', 'ARR_DELAY']
data_1989 = data_1989[variables]
data_1989 = data_1989.dropna()

In [10]:
## Train test split or DT
X_variables_dt = ['DAY_OF_WEEK', 'SEASON', 'DISTANCE', 'CRS_ARR_BIN', 
               'PRCP_ORIGIN', 'SNOW_ORIGIN', 'SNWD_ORIGIN', 'TMAX_ORIGIN', 'TMIN_ORIGIN']
X_data_dt = data_1989[X_variables_dt]
X_data_dt = pd.get_dummies(X_data_dt, columns = ['DAY_OF_WEEK', 'CRS_ARR_BIN', 'SEASON'])
y_data_dt = data_1989['HAS_ARR_DELAY']
X_train_dt, X_test_dt, y_train_dt, y_test_dt = train_test_split(X_data_dt, y_data_dt, test_size = 0.2)

In [11]:
## Decision Tree model 
dt = DecisionTreeClassifier()
dt.fit(X_train_dt, y_train_dt)

DecisionTreeClassifier()

In [12]:
new_y = dt.predict(X_test_dt)
print(balanced_accuracy_score(new_y, y_test_dt))

0.6321804219054523


In [13]:
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_data_dt, y_data_dt)
X_train_dt, X_test_dt, y_train_dt, y_test_dt = train_test_split(X_resampled, y_resampled, test_size = 0.2)

In [14]:
dt_arr = DecisionTreeClassifier()
dt_arr.fit(X_train_dt, y_train_dt)
sample_y = dt_arr.predict(X_test_dt)
print(accuracy_score(sample_y, y_test_dt))

0.6846353734726599
